In [1]:
import time
import csv

# CLP =================================================================================================================================
def clp(graf):
    zac = time.time() #Zacetek merjenja casa

    CLP = MixedIntegerLinearProgram(maximization = True) #CLP, ki ga bomo maximizirali
    vozlisca = CLP.new_variable(binary = True) #Vsakemu vozliscu priredimo vrednost 1 ali 0 (binarne spremenljivke)
    CLP.set_objective(sum([vozlisca[v] for v in graf.vertices()])) #Definiramo kriterijsko funkcijo

    for u,v in graf.edges(labels = False):
        CLP.add_constraint(vozlisca[u] + vozlisca[v] <= 1) #Prvi pogoj CLP

    CLP.solve() #resi CLP
    vozlisca = CLP.get_values(vozlisca) #za vsako vozlisce dobimo njegovo vrednost

    #Moc mnozice I:
    st = 0
    for i in vozlisca:
        st = st + vozlisca[i]

    #Seznam vozlisc v mnozici I:
    sez = []
    for i in vozlisca:
        if vozlisca[i] == 1.0:
            sez.append(i)

    #Merjenje casa:
    kon = time.time()
    cas = kon - zac
    return [st, sez, cas]


# Relaksacija CLP ==========================================================================================================================
def rclp(graf):
    zac = time.time() #Zacetek merjenja casa

    RCLP = MixedIntegerLinearProgram(maximization = True) #Relak. CLP, ki ga bomo maximizirali
    vozlisca_RCLP = RCLP.new_variable(real = True) #Vsakemu vozliscu priredimo vrednost 1 ali 0
    RCLP.set_max(vozlisca_RCLP,1)
    RCLP.set_min(vozlisca_RCLP,0)
    RCLP.set_objective(sum([vozlisca_RCLP[v] for v in graf.vertices()])) #Kriterijska funkcija

    for u,v in graf.edges(labels = False):
        RCLP.add_constraint(vozlisca_RCLP[u] + vozlisca_RCLP[v] <= 1) #Prvi pogoj 

    RCLP.solve()
    vozlisca_RCLP = RCLP.get_values(vozlisca_RCLP)

    #Moc mnozice I: %% ALI JE TO SPLOH SMISELNO? %%
    st = 0
    for i in vozlisca_RCLP:
        st = st + vozlisca_RCLP[i]

    #Merjenje casa:
    kon = time.time()
    cas = kon - zac
    return [st, None, cas]


# Lokalno iskanje ==========================================================================================================================
def lokalno_iskanje(graf, st_ponovitev):
    moc_max = 0 #moc največje neodvisne mnozice vozlisc
    max_seznam_neod = [] #največja mnozica neodvisnih vozlisc
    zac = time.time()

    for i in range(st_ponovitev):
        seznam_neod = [] #seznam neodvisnih vozlisc
        seznam_neod.append(graf.random_vertex()) #dodamo nakljucno vozlisce v seznam_neod
        for v in graf.vertices():
            if v in seznam_neod: #ce je v ze v seznam_neod, nadaljujemo
                continue
            else:
                indikator = 0
                for i in seznam_neod:
                    if (i,v) in graf.edges(labels=False) or (v,i) in graf.edges(labels=False):
                        indikator = 1
                if indikator == 0:
                    seznam_neod.append(v) #ce ni povezave med vozlisci v seznam_neod in v, v dodamo v seznam_neod
        if len(seznam_neod) > moc_max: #iscemo najvecjo mnozico neodvisnih vozlisc
            max_seznam_neod = seznam_neod
            moc_max = len(seznam_neod)
    #Merjenje casa:
    kon = time.time()
    cas = kon - zac
    max_seznam_neod.sort() #Ureditev max mnozice neodvisnih vozlisc
    return[moc_max, max_seznam_neod, cas]

In [2]:
# Funkcija za testiranje vseh treh algoritmov na nakljucnem grafu:
# testiraj(stevilo vozlisc, verjetnost povezave med dvema vozliscema, stevilo ponovitev lokalnega iskanja) =======================================
def testiraj(st_vozlisc, verjetnost, st_ponovitev=10):
    rezultati = []
    graf = graphs.RandomGNP(st_vozlisc, verjetnost)
    rezultati.append(clp(graf))
    rezultati.append(rclp(graf))
    rezultati.append(lokalno_iskanje(graf, st_ponovitev))
    return rezultati

In [3]:
testiraj(10,0.2,30)

[[6.0, [2, 4, 5, 6, 8, 9], 0.0377810001373291],
 [6.0, None, 0.0020787715911865234],
 [6, [2, 4, 5, 6, 8, 9], 0.017839908599853516]]

In [4]:
# Funkcija za izapis v CSV (izvede zgornjo funkcijo 'testiraj' na grafih s številom vozlisc od 1 do max_stevilo_vozlisc, pri dolocenih verjetnostih povezave dveh vozlisc in dolocenem stevilu ponovitev lokalnega iskanja):
def izpis_csv(max_stevilo_vozlisc, verjetnost, st_ponovitev):
    trenutni_cas = time.strftime("%Y-%m-%d_%H-%M-%S")
    with open('osebe_trenutni_cas.csv', 'wb') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)

        filewriter.writerow([['CLP_MAX_MOC', 'CLP_VOZLISCA_I','CLP_CAS'], ['RCLP_MAX_MOC', 'Neuporabno','RCLP_CAS'], ['LOKISK_MAX_MOC', 'LOKISK_VOZLISCA_I','LOKISK_CAS']])
                             
        for i in range(1, max_stevilo_vozlisc + 1):
                             filewriter.writerow(testiraj(i, verjetnost, st_ponovitev))


In [5]:
izpis_csv(8,0.3,5)